<a href="https://colab.research.google.com/github/cnboonhan/colab/blob/main/mixtral.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pkill ollama
! curl https://ollama.ai/install.sh | sh
! pip install llama-index qdrant_client torch transformers pyTelegramBotAPI

import subprocess
subprocess.Popen(["ollama", "serve"])
! ollama pull mistral

In [ ]:
from pathlib import Path
import qdrant_client
from llama_index import (
    VectorStoreIndex,
    ServiceContext,
    download_loader,
)
from llama_index.llms import Ollama
from llama_index.storage.storage_context import StorageContext
from llama_index.vector_stores.qdrant import QdrantVectorStore

llm = Ollama(model="mistral")

JSONReader = download_loader("JSONReader")
loader = JSONReader()
documents = loader.load_data(Path('./tinytweets.json'))
client = qdrant_client.QdrantClient(
    path="./qdrant_data"
)
vector_store = QdrantVectorStore(client=client, collection_name="tweets")
storage_context = StorageContext.from_defaults(vector_store=vector_store)
service_context = ServiceContext.from_defaults(llm=llm,embed_model="local")
index = VectorStoreIndex.from_documents(documents,service_context=service_context,storage_context=storage_context)
query_engine = index.as_query_engine()

In [ ]:
import telebot
key = input("Provide Telegram Bot Key")
bot = telebot.TeleBot(key, parse_mode=None)
@bot.message_handler(func=lambda m: True)
def echo_all(message):
  response = query_engine.query(message.text)
  bot.reply_to(message, response)
bot.infinity_polling()